In [6]:
from pydantic import BaseModel

class Vector3(BaseModel):
    x: float
    y: float
    z: float

class SceneObject(BaseModel):
    name: str
    position: Vector3
    size: Vector3

class Scene(BaseModel):
    objects: list[SceneObject]

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.responses.parse(
    model="gpt-4o-2024-08-06",
    input=[
        {
            "role": "system",
            "content": "Imagine a scene based on the prompt with some objects in the scene. Extract the scene information.",
        },
        {
            "role": "user",
            "content": "Space",
        },
    ],
    text_format=Scene,
)

scene = response.output_parsed
print(scene)

In [14]:
scene_json = scene.model_dump_json(indent=4)
with open("./Unity/AIML Research Project/Assets/scene.json", "w") as scene_file:
    scene_file.write(scene_json)

In [15]:
import re
import subprocess
import tempfile
from pathlib import Path

import torch
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.diffusion.sample import sample_latents
from shap_e.models.download import load_config, load_model
from shap_e.util.notebooks import decode_latent_mesh

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xm = load_model("transmitter", device=device)
model = load_model("text300M", device=device)
diffusion = diffusion_from_config(load_config("diffusion"))

batch_size = 1  # this is the size of the models, higher values take longer to generate.
guidance_scale = 15.0  # this is the scale of the guidance, higher values make the model look more like the prompt.

output_dir = Path("./Unity/AIML Research Project/Assets/GLB")
output_dir.mkdir(parents=True, exist_ok=True)

# Generate a safe filename from the prompt
def slugify(text: str) -> str:
    return re.sub(r"[^a-zA-Z0-9\-]+", "-", text.strip().lower()).strip("-")

for object in scene.objects:
    object_name = object.name

    base_name = slugify(object_name)
    output_file = output_dir / f"{base_name}.glb"

    print(f"[+] Generating model for prompt: '{object_name}'")
    latents = sample_latents(
        batch_size=batch_size,
        model=model,
        diffusion=diffusion,
        guidance_scale=guidance_scale,
        model_kwargs=dict(texts=[object_name] * batch_size),
        progress=True,
        clip_denoised=True,
        use_fp16=True,
        use_karras=True,
        karras_steps=64,
        sigma_min=1e-3,
        sigma_max=160,
        s_churn=0,
    )

    with tempfile.TemporaryDirectory() as tmpdir:
        tmp_path = Path(tmpdir)
        ply_file = tmp_path / "mesh.ply"

        print(f"[+] Saving intermediate PLY to: {ply_file}")
        t = decode_latent_mesh(xm, latents[0]).tri_mesh()
        with open(ply_file, "wb") as f:
            t.write_ply(f)

        # Call Blender to convert PLY → GLB
        blender_script = Path("./Blender/export.py").resolve()
        command = [
            "blender",
            "-b",
            "-P",
            str(blender_script),
            "--",
            str(ply_file),
            str(output_file),
        ]
        print(f"[+] Running Blender export script...")
        subprocess.run(command, check=True)

    print(f"[✓] Exported: {output_file}")

[+] Generating model for prompt: 'Astronaut'


  0%|          | 0/64 [00:00<?, ?it/s]

[+] Saving intermediate PLY to: /tmp/tmpgs7avhvt/mesh.ply


/home/chris/repos/COMP-SCI-7205-AIML-Research-Project/.venv/lib/python3.10/site-packages/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/home/chris/repos/COMP-SCI-7205-AIML-Research-Project/.venv/lib/python3.10/site-packages/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


[+] Running Blender export script...
Color management: using fallback mode for management
Color management: Error could not find role data role.
Blender 3.0.1
Color management: scene view "Filmic" not found, setting default "Standard".
/run/user/1000/gvfs/ non-existent directory
Color management: scene view "Filmic" not found, setting default "Standard".

Successfully imported '/tmp/tmpgs7avhvt/mesh.ply' in 0.447 sec
'/usr/bin/3.0/python/lib/python3.10/site-packages/libextern_draco.so' does not exist, draco mesh compression not available, please add it or create environment variable BLENDER_EXTERN_DRACO_LIBRARY_PATH pointing to the folder
11:15:42 | INFO: Starting glTF 2.0 export
11:15:42 | INFO: Extracting primitive: mesh
11:15:42 | INFO: Primitives created: 1
11:15:42 | INFO: Finished glTF 2.0 export in 0.15267610549926758 s

Exported Unity/AIML Research Project/Assets/GLB/astronaut.glb successfully.

Blender quit
[✓] Exported: Unity/AIML Research Project/Assets/GLB/astronaut.glb
[+]

...

[+] Running Blender export script...
Color management: using fallback mode for management
Color management: Error could not find role data role.
Blender 3.0.1
Color management: scene view "Filmic" not found, setting default "Standard".
/run/user/1000/gvfs/ non-existent directory
Color management: scene view "Filmic" not found, setting default "Standard".

Successfully imported '/tmp/tmpsg7lqvpe/mesh.ply' in 0.787 sec
'/usr/bin/3.0/python/lib/python3.10/site-packages/libextern_draco.so' does not exist, draco mesh compression not available, please add it or create environment variable BLENDER_EXTERN_DRACO_LIBRARY_PATH pointing to the folder
11:16:33 | INFO: Starting glTF 2.0 export
11:16:33 | INFO: Extracting primitive: mesh
11:16:33 | INFO: Primitives created: 1
11:16:33 | INFO: Finished glTF 2.0 export in 0.5705184936523438 s

Exported Unity/AIML Research Project/Assets/GLB/space-station.glb successfully.

Blender quit
[✓] Exported: Unity/AIML Research Project/Assets/GLB/space-station.

  0%|          | 0/64 [00:00<?, ?it/s]

[+] Saving intermediate PLY to: /tmp/tmpw8_udvds/mesh.ply


/home/chris/repos/COMP-SCI-7205-AIML-Research-Project/.venv/lib/python3.10/site-packages/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/home/chris/repos/COMP-SCI-7205-AIML-Research-Project/.venv/lib/python3.10/site-packages/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


[+] Running Blender export script...
Color management: using fallback mode for management
Color management: Error could not find role data role.
Blender 3.0.1
Color management: scene view "Filmic" not found, setting default "Standard".
/run/user/1000/gvfs/ non-existent directory
Color management: scene view "Filmic" not found, setting default "Standard".

Successfully imported '/tmp/tmpw8_udvds/mesh.ply' in 0.461 sec
'/usr/bin/3.0/python/lib/python3.10/site-packages/libextern_draco.so' does not exist, draco mesh compression not available, please add it or create environment variable BLENDER_EXTERN_DRACO_LIBRARY_PATH pointing to the folder
11:17:17 | INFO: Starting glTF 2.0 export
11:17:17 | INFO: Extracting primitive: mesh
11:17:17 | INFO: Primitives created: 1
11:17:17 | INFO: Finished glTF 2.0 export in 0.14284849166870117 s

Exported Unity/AIML Research Project/Assets/GLB/space-shuttle.glb successfully.

Blender quit
[✓] Exported: Unity/AIML Research Project/Assets/GLB/space-shuttle